In [2]:
############################################################
####
#####由higahsi生成的原始的稀疏矩阵生成prebulk数据及bed文件作为dchic的输入
#####
##########################################################
#############################################################
#############################################################
##################################################################
##################################################################
from Higashi_backend.utils import *
from Higashi_analysis.Higashi_analysis import *
import h5py
from sklearn.preprocessing import MinMaxScaler, quantile_transform
import os

os.environ["OMP_NUM_THREADS"] = "10"
import pandas as pd
import argparse
import scipy.sparse
from scipy.sparse import hstack,vstack
import math

config=get_config('/home/python/higashi/dataset2/cortex_250k_tmp/config_cellcycle.json')
res = config['resolution']
data_dir = config['data_dir']
temp_dir = config['temp_dir']
raw_dir = os.path.join(temp_dir, "raw")
if 'cytoband_path' in config:
	cytoband_path = config['cytoband_path']
else:
	cytoband_path = None
neighbor_num = config['neighbor_num']
embedding_name = config['embedding_name']

chrom_list = config['impute_list']
# chrom_list = ['chr1', 'chr2']
chrom_list = np.array(chrom_list)


################从基因组参考文件计算一定分辨率下各染色体bin的数量
def create_bed(res):

    file_path = '/home/python/higashi/dataset2/cortex/config/mm10.chrom.sizes.txt'  
    with open(file_path, 'r') as file:
        lines = file.readlines()
    chromosome_bins = {}
    max_lens = {}
    # 特定分辨率
    resolution = res # 请替换成你的分辨率
    # 遍历文件中的每一行
    for line in lines:
        # 分割每一行的染色体和对应的长度
        chromosome, length = line.strip().split('\t')
        
        # 计算染色体对应的bin数量
        # bin_count = int(length) // resolution
        bin_count = math.ceil(int(length) / resolution)
        
        # 存储结果到字典中
        chromosome_bins[chromosome] = bin_count
        max_lens[chromosome] = int(length)
    bin_chrom_list = []
    bin_start_list = []
    bin_end_list = []
    for chrom in chromosome_bins:
        bin_count = chromosome_bins[chrom]
        bin_chrom_list+=[chrom]*bin_count
        bin_start_list.extend((np.arange(bin_count) * res))
        end=(np.arange(bin_count) + 1) * res
        # print(len(end))
        max_len=max_lens[chromosome]
        end[bin_count-1]=max_len
        # bin_end_list.extend(((np.arange(bin_count) + 1) * res))
        bin_end_list.extend(end)
        # print (chrom, "finished")
        # print(chrom)
        # print(bin_count)
    index_list=np.arange(1,len(bin_chrom_list)+1)
    df=pd.DataFrame()
    df['chr']=bin_chrom_list
    df['start']=bin_start_list
    df['end']=bin_end_list
    df['index']=index_list
    bed_path='test_bed.bed'
    df.to_csv(bed_path, sep='\t', index=None, header=False, mode='w')
    print(f"bed file written to {bed_path}")
    return chromosome_bins


def process_chrom(chrom,cell_type):
# Get the raw sparse mtx list
    origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)
    size = origin_sparse[0].shape[0]
    pre_bulk_list=[]
    # sum_by_label=[]
    a={}
    b={}
    # c={}
    for j in list(set(cell_type)):
        # print(j)
        indices = [index for index, value in enumerate(cell_type) if value == j]
        a[j]=indices
        b[j] = np.zeros_like(np.array(origin_sparse[0]))
        for i in a[j]:
            proba = np.array(origin_sparse[i])    
            b[j] +=proba
        temp = np.array(b[j].item().todense())
        temp=scipy.sparse.csr_matrix(temp)
            # temp = b[j]
        pre_bulk_list.append(temp)
    return pre_bulk_list




In [3]:
from tqdm import tqdm
output_dir='/home/python/higashi/dcpc/cortex_prebulk_from_sc_with_rep'
print('chrom_list:',chrom_list)
# with open('matrix_values.txt', 'w') as file:
# cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
#使用重复样本标签

cell_type_info=pickle.load(open('/home/python/higashi/notebook/cortex_label_info.pickle', "rb"))
cell_type=cell_type_info['cell_type']
cell_type_list=[]
chromosome_bins=create_bed(res)
for j in list(set(cell_type)):
    cell_type_list.append(j)
bin_count=0
pre_bulk_raw=[]
for chrom in chrom_list:
    pre_bulk_list=process_chrom(chrom,cell_type)
    print (chrom, "finished")
    pre_bulk_list_unzip=[]
    for bulk in pre_bulk_list:
        rows, cols, values = scipy.sparse.find(bulk)
        rows+=bin_count
        cols+=bin_count
        pre_bulk_list_unzip.append([cols,rows,values])
        # pre_bulk_list_unzip.append(rows)
        # pre_bulk_list_unzip.append(values)
        # bulk.nonzero().row+=bin_count
        # bulk.nonzero().col+=bin_count
    bin_count += chromosome_bins[chrom] 
    pre_bulk_raw.append(pre_bulk_list_unzip)
temp_bulk ={}
for i in tqdm(range(len(pre_bulk_raw[0]))):
    sample_name=cell_type_list[i]+'.matrix'
    save_path=os.path.join(output_dir, sample_name)
    # save_path=cell_type_list[i]+'_prebulk_mat.txt'   
    temp_bulk[i]=[]
    for j in range(len(pre_bulk_raw)):
        # print('j,i',j,i)
        temp_bulk[i].append(pre_bulk_raw[j][i])
    temp_bulk[i]=np.column_stack([np.vstack(group) for group in temp_bulk[i]])
    np.savetxt(save_path, temp_bulk[i].T, delimiter='\t', fmt='%d')
    print(f"Data written to {save_path}")








#     pre_bulk_raw.append(pre_bulk_list)
# data_bulk_raw=[]
# temp_bulk={}

# for i in range(len(pre_bulk_raw[0])):
#     save_path=cell_type_list[i]+'_prebulk_mat.txt'
#     with open(save_path, 'w') as file:
#         temp_bulk[i]=[]
#         for j in range(len(pre_bulk_raw)):
#             print('j,i',j,i)
#             temp_bulk[i].append(pre_bulk_raw[j][i])

#         for matrix in temp_bulk[i]:
#             for row, col, value in zip(*matrix.nonzero(), matrix.data):
#                 file.write(f"{row}\t{col}\t{value}\n")



chrom_list: ['chr1' 'chr2' 'chr3' 'chr4' 'chr5' 'chr6' 'chr7' 'chr8' 'chr9' 'chr10'
 'chr11' 'chr12' 'chr13' 'chr14' 'chr15' 'chr16' 'chr17' 'chr18' 'chr19']
bed file written to test_bed.bed
chr1 finished
chr2 finished
chr3 finished
chr4 finished
chr5 finished
chr6 finished
chr7 finished
chr8 finished
chr9 finished
chr10 finished
chr11 finished
chr12 finished
chr13 finished
chr14 finished
chr15 finished
chr16 finished
chr17 finished
chr18 finished
chr19 finished


 17%|█▋        | 1/6 [00:11<00:55, 11.08s/it]

Data written to /home/python/higashi/dcpc/cortex_prebulk_from_sc_with_rep/cortex-p347-cb_250Kb.matrix


 33%|███▎      | 2/6 [00:21<00:42, 10.59s/it]

Data written to /home/python/higashi/dcpc/cortex_prebulk_from_sc_with_rep/cortex-p007-cb_250Kb.matrix


...

Data written to /home/python/higashi/dcpc/cortex_prebulk_from_sc_with_rep/cortex-p056-cb_250Kb.matrix


 83%|████████▎ | 5/6 [00:56<00:11, 11.54s/it]

Data written to /home/python/higashi/dcpc/cortex_prebulk_from_sc_with_rep/cortex-p309-cb_250Kb.matrix


100%|██████████| 6/6 [01:08<00:00, 11.41s/it]

Data written to /home/python/higashi/dcpc/cortex_prebulk_from_sc_with_rep/cortex-p0028-cb_250Kb.matrix


In [15]:
cell_type_info=pickle.load(open('/home/python/higashi/notebook/cortex_label_info.pickle', "rb"))
cell_type=cell_type_info['cell_type']
cell_type_list=[]
print(len(cell_type))
chromosome_bins=create_bed(res)
for j in list(set(cell_type)):
    cell_type_list.append(j)
print(len(cell_type_list))

1075
bed file written to test_bed.bed
6


In [282]:
# file_path = '/home/python/higashi/dataset2/cortex/config/mm10.chrom.sizes.txt'  
# with open(file_path, 'r') as file:
#     lines = file.readlines()
# chromosome_bins = {}
# # 特定分辨率
# resolution = res # 请替换成你的分辨率
# max_len={}
# # 遍历文件中的每一行
# for line in lines:
#     # 分割每一行的染色体和对应的长度
#     chromosome, length = line.strip().split('\t')
    
#     # 计算染色体对应的bin数量
#     # bin_count = int(length) // resolution
#     bin_count = math.ceil(int(length) / resolution)
#     # print(bin_count)
    
#     # 存储结果到字典中
#     chromosome_bins[chromosome] = bin_count
#     max_len[chromosome]=int(length)


782
729
685
641
627
608
599
582
523
518
500
499
489
482
481
417
393
380
367
363
246
4
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [270]:
chrom_list

array(['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
       'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19'], dtype='<U5')

In [286]:

# bin_chrom_list = []
# bin_start_list = []
# bin_end_list = []
# for chrom in chromosome_bins:
#     bin_count = chromosome_bins[chrom]
#     bin_chrom_list+=[chrom]*bin_count
#     bin_start_list.extend((np.arange(bin_count) * res))
#     end=(np.arange(bin_count) + 1) * res
#     # print(len(end))
#     end[bin_count-1]=max_len[chrom]
#     # bin_end_list.extend(((np.arange(bin_count) + 1) * res))
#     bin_end_list.extend(end)
#     # print (chrom, "finished")
#     # print(chrom)
#     # print(bin_count)
# index_list=np.arange(1,len(bin_chrom_list)+1)
# df=pd.DataFrame()
# df['chr']=bin_chrom_list
# df['start']=bin_start_list
# df['end']=bin_end_list
# df['index']=index_list
# bed_path='test_bed.bed'
# df.to_csv(bed_path, sep='\t', index=None, header=False, mode='w')
# print(f"bed file written to {bed_path}")




chr1 finished
chr2 finished
chrX finished
chr3 finished
chr4 finished
chr5 finished
chr6 finished
chr7 finished
chr10 finished
chr8 finished
chr14 finished
chr9 finished
chr11 finished
chr13 finished
chr12 finished
chr15 finished
chr16 finished
chr17 finished
chrY finished
chr18 finished
chr19 finished
chr5_JH584299_random finished
chrX_GL456233_random finished
chrY_JH584301_random finished
chr1_GL456211_random finished
chr4_GL456350_random finished
chr4_JH584293_random finished
chr1_GL456221_random finished
chr5_JH584297_random finished
chr5_JH584296_random finished
chr5_GL456354_random finished
chr4_JH584294_random finished
chr5_JH584298_random finished
chrY_JH584300_random finished
chr7_GL456219_random finished
chr1_GL456210_random finished
chrY_JH584303_random finished
chrY_JH584302_random finished
chr1_GL456212_random finished
chrUn_JH584304 finished
chrUn_GL456379 finished
chr4_GL456216_random finished
chrUn_GL456393 finished
chrUn_GL456366 finished
chrUn_GL456367 finished
chrUn_

In [273]:
len(bin_start_list)

10961

In [252]:
for i in range(1,783):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [285]:
index_list=np.arange(1,5)
index_list

array([1, 2, 3, 4])

In [276]:
a=np.arange(4)*1000
a

array([   0, 1000, 2000, 3000])

In [277]:
a[3]=5

In [278]:
a

array([   0, 1000, 2000,    5])

In [259]:
a=np.arange(1,782)*1000
a


array([  1000,   2000,   3000,   4000,   5000,   6000,   7000,   8000,
         9000,  10000,  11000,  12000,  13000,  14000,  15000,  16000,
        17000,  18000,  19000,  20000,  21000,  22000,  23000,  24000,
        25000,  26000,  27000,  28000,  29000,  30000,  31000,  32000,
        33000,  34000,  35000,  36000,  37000,  38000,  39000,  40000,
        41000,  42000,  43000,  44000,  45000,  46000,  47000,  48000,
        49000,  50000,  51000,  52000,  53000,  54000,  55000,  56000,
        57000,  58000,  59000,  60000,  61000,  62000,  63000,  64000,
        65000,  66000,  67000,  68000,  69000,  70000,  71000,  72000,
        73000,  74000,  75000,  76000,  77000,  78000,  79000,  80000,
        81000,  82000,  83000,  84000,  85000,  86000,  87000,  88000,
        89000,  90000,  91000,  92000,  93000,  94000,  95000,  96000,
        97000,  98000,  99000, 100000, 101000, 102000, 103000, 104000,
       105000, 106000, 107000, 108000, 109000, 110000, 111000, 112000,
      

In [238]:
a=[]
a.append(pre_bulk_raw[0][1])
a.append(pre_bulk_raw[0][2])
a= np.column_stack([np.vstack(group) for group in a])
# b=np.concatenate(a)

In [239]:
a

array([[  12.,   12.,   12., ...,  781.,  781.,  781.],
       [  12.,   13.,   14., ...,  779.,  780.,  781.],
       [7912., 3804., 1960., ...,   96.,  932., 1388.]])

In [135]:
temp_bulk

{0: [<782x782 sparse matrix of type '<class 'numpy.float32'>'
  	with 497258 stored elements in Compressed Sparse Row format>,
  <729x729 sparse matrix of type '<class 'numpy.float32'>'
  	with 424834 stored elements in Compressed Sparse Row format>],
 1: [<782x782 sparse matrix of type '<class 'numpy.float32'>'
  	with 531546 stored elements in Compressed Sparse Row format>,
  <729x729 sparse matrix of type '<class 'numpy.float32'>'
  	with 435806 stored elements in Compressed Sparse Row format>],
 2: [<782x782 sparse matrix of type '<class 'numpy.float32'>'
  	with 503676 stored elements in Compressed Sparse Row format>,
  <729x729 sparse matrix of type '<class 'numpy.float32'>'
  	with 425912 stored elements in Compressed Sparse Row format>]}

In [117]:
type(pre_bulk_list[0])

scipy.sparse.csr.csr_matrix

In [120]:
print(len(pre_bulk_raw[0]))

3


In [116]:
type(pre_bulk_raw[0][0])

scipy.sparse.csr.csr_matrix

In [107]:
data_bulk_raw

[<782x2346 sparse matrix of type '<class 'numpy.float32'>'
 	with 1532480 stored elements in COOrdinate format>,
 <729x2187 sparse matrix of type '<class 'numpy.float32'>'
 	with 1286552 stored elements in COOrdinate format>]

array([<782x782 sparse matrix of type '<class 'numpy.float32'>'
       	with 497258 stored elements in Compressed Sparse Row format>,
       <782x782 sparse matrix of type '<class 'numpy.float32'>'
       	with 531546 stored elements in Compressed Sparse Row format>,
       <782x782 sparse matrix of type '<class 'numpy.float32'>'
       	with 503676 stored elements in Compressed Sparse Row format>,
       <729x729 sparse matrix of type '<class 'numpy.float32'>'
       	with 424834 stored elements in Compressed Sparse Row format>,
       <729x729 sparse matrix of type '<class 'numpy.float32'>'
       	with 435806 stored elements in Compressed Sparse Row format>,
       <729x729 sparse matrix of type '<class 'numpy.float32'>'
       	with 425912 stored elements in Compressed Sparse Row format>],
      dtype=object)

In [118]:
for i in range(3):
    print(i)

0
1
2


In [102]:
pre_bulk_raw

[[<782x782 sparse matrix of type '<class 'numpy.float32'>'
  	with 497258 stored elements in Compressed Sparse Row format>,
  <782x782 sparse matrix of type '<class 'numpy.float32'>'
  	with 531546 stored elements in Compressed Sparse Row format>,
  <782x782 sparse matrix of type '<class 'numpy.float32'>'
  	with 503676 stored elements in Compressed Sparse Row format>],
 [<729x729 sparse matrix of type '<class 'numpy.float32'>'
  	with 424834 stored elements in Compressed Sparse Row format>,
  <729x729 sparse matrix of type '<class 'numpy.float32'>'
  	with 435806 stored elements in Compressed Sparse Row format>,
  <729x729 sparse matrix of type '<class 'numpy.float32'>'
  	with 425912 stored elements in Compressed Sparse Row format>]]

In [89]:
a_mat=scipy.sparse.csr_matrix(pre_bulk_list[0])

In [90]:
type(a_mat)

scipy.sparse.csr.csr_matrix

In [94]:
len(a_mat.nonzero()[0])

424834

In [12]:
import scipy.sparse

In [28]:
origin_sparse[4].shape

(729, 729)

In [23]:
pre_bulk_list[2].shape

(729, 729)

In [13]:
test_mat=scipy.sparse.csr_matrix(pre_bulk_list[0])

In [22]:
print(test_mat)

  (12, 12)	7688.0
  (12, 13)	4084.0
  (12, 14)	386.0
  (12, 15)	308.0
  (12, 16)	172.0
  (12, 17)	140.0
  (12, 18)	162.0
  (12, 19)	180.0
  (12, 20)	24.0
  (12, 21)	32.0
  (12, 22)	44.0
  (12, 23)	14.0
  (12, 24)	26.0
  (12, 25)	14.0
  (12, 26)	22.0
  (12, 27)	10.0
  (12, 28)	18.0
  (12, 29)	6.0
  (12, 30)	16.0
  (12, 31)	20.0
  (12, 32)	10.0
  (12, 33)	12.0
  (12, 34)	10.0
  (12, 35)	10.0
  (12, 36)	20.0
  :	:
  (728, 669)	4.0
  (728, 670)	4.0
  (728, 675)	2.0
  (728, 679)	2.0
  (728, 680)	2.0
  (728, 687)	2.0
  (728, 689)	2.0
  (728, 692)	2.0
  (728, 707)	2.0
  (728, 708)	4.0
  (728, 710)	2.0
  (728, 711)	4.0
  (728, 713)	2.0
  (728, 715)	6.0
  (728, 716)	2.0
  (728, 719)	2.0
  (728, 720)	4.0
  (728, 721)	4.0
  (728, 722)	2.0
  (728, 723)	4.0
  (728, 724)	14.0
  (728, 725)	14.0
  (728, 726)	40.0
  (728, 727)	378.0
  (728, 728)	122.0


In [29]:
bulk1 = np.array(np.sum(origin_sparse, axis=0))

In [74]:
chrom='chr2'
# Get the raw sparse mtx list
origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)
type(origin_sparse[0])

(246, 246)

In [72]:
print((bulk1.item()).row)

AttributeError: row not found

In [66]:
print(bulk1)

  (12, 12)	26912.0
  (12, 13)	10456.0
  (12, 14)	1184.0
  (12, 15)	854.0
  (12, 16)	516.0
  (12, 17)	424.0
  (12, 18)	498.0
  (12, 19)	466.0
  (12, 20)	160.0
  (12, 21)	128.0
  (12, 22)	132.0
  (12, 23)	134.0
  (12, 24)	216.0
  (12, 25)	152.0
  (12, 26)	122.0
  (12, 27)	114.0
  (12, 28)	76.0
  (12, 29)	86.0
  (12, 30)	52.0
  (12, 31)	68.0
  (12, 32)	80.0
  (12, 33)	72.0
  (12, 34)	80.0
  (12, 35)	82.0
  (12, 36)	92.0
  :	:
  (728, 697)	10.0
  (728, 698)	6.0
  (728, 699)	10.0
  (728, 706)	2.0
  (728, 707)	2.0
  (728, 708)	10.0
  (728, 709)	16.0
  (728, 710)	16.0
  (728, 711)	12.0
  (728, 712)	10.0
  (728, 713)	6.0
  (728, 715)	20.0
  (728, 716)	12.0
  (728, 717)	6.0
  (728, 718)	6.0
  (728, 719)	6.0
  (728, 720)	10.0
  (728, 721)	18.0
  (728, 722)	14.0
  (728, 723)	24.0
  (728, 724)	20.0
  (728, 725)	38.0
  (728, 726)	124.0
  (728, 727)	950.0
  (728, 728)	430.0


In [36]:
a_list=[['chr']*5,['chr']*6,['chr']*9]

len(a_list[0])

5

In [38]:
bin_list=[]
for a in a_list:
    # print(a)
    length = len(a)
    bin_list.append((np.arange(length)).astype('int'))
    # bin_chrom_list += [chrom] * len(use_rows)
    # bin_start_list.append((np.arange(length) * res).astype('int')[use_rows])
    # bin_end_list.append(((np.arange(length) + 1) * res).astype('int')[use_rows])
    # bulk_cp_all.append(bulk_compartment)

In [40]:
data=np.concatenate(bin_list)

In [41]:
data

array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8])

In [62]:
bin_list=[]
size=0
for chrom in chrom_list:
    origin_sparse= np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)
    size += origin_sparse[0].shape[0]
    print(chrom,size)
    bin_list.append((np.arange(size)+1).astype('int'))

bin_all=np.concatenate(bin_list)   

    

chr1 782
chr2 1511
chr3 2152
chr4 2779
chr5 3387
chr6 3986
chr7 4568
chr8 5086
chr9 5585
chr10 6108
chr11 6597
chr12 7078
chr13 7560
chr14 8060
chr15 8477
chr16 8870
chr17 9250
chr18 9613
chr19 9859


In [63]:
bin_all

array([   1,    2,    3, ..., 9857, 9858, 9859])

In [39]:
bin_list

[array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4, 5]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8])]